In [91]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline




In [92]:
df = pd.read_csv(r"C:\Users\T PLUG\Downloads\playground-series-s5e7\train.csv")
df.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert


In [93]:

# # Step 2: Preprocess the data
# # Convert categorical to numeric
# le = LabelEncoder()
# imputer = SimpleImputer(strategy='mean')

# def wrangle_data(df, fit=True):
#     df = df.copy()

#     # Encode categorical columns
#     if fit:
#         df['Stage_fear'] = le.fit_transform(df['Stage_fear'])
#         df['Drained_after_socializing'] = le.fit_transform(df['Drained_after_socializing'].astype(str))
#         df['Personality'] = le.fit_transform(df['Personality'])  # Extrovert=1, Introvert=0
#     else:
#         df['Stage_fear'] = le.transform(df['Stage_fear'])
#         df['Drained_after_socializing'] = le.transform(df['Drained_after_socializing'].astype(str))
#         df['Personality'] = le.transform(df['Personality'])  # Extrovert=1, Introvert=0

#     features_df = df.drop(columns=[col for col in ['id', 'Personality'] if col in df.columns])
    
#     # Impute missing values
#     if fit:
#         df_imputed = pd.DataFrame(imputer.fit_transform(features_df), columns=features_df.columns)
#     else:
#         df_imputed = pd.DataFrame(imputer.transform(features_df), columns=features_df.columns)

#     return df_imputed



In [94]:
df['Personality'].value_counts(normalize=True)

Extrovert    0.739527
Introvert    0.260473
Name: Personality, dtype: float64

In [95]:
df.corr()

,id,Time_spent_Alone,Social_event_attendance,Going_outside,Friends_circle_size,Post_frequency
id,1.000000,-0.000004,0.002750,0.004460,0.004340,-0.001945
Time_spent_Alone,-0.000004,1.000000,-0.628806,-0.640884,-0.598014,-0.611544
Social_event_attendance,0.002750,-0.628806,1.000000,0.585224,0.566675,0.566679
Going_outside,0.004460,-0.640884,0.585224,1.000000,0.549864,0.579305
Friends_circle_size,0.004340,-0.598014,0.566675,0.549864,1.000000,0.522272
Post_frequency,-0.001945,-0.611544,0.566679,0.579305,0.522272,1.000000


In [96]:
# Encoders and imputer stored globally
stage_fear_encoder = LabelEncoder()
drained_encoder = LabelEncoder()
imputer = SimpleImputer(strategy='mean')

def wrangle_data(df, fit=True):
    df = df.copy()

    if 'Stage_fear' in df.columns:
        if fit:
            df['Stage_fear'] = stage_fear_encoder.fit_transform(df['Stage_fear'].astype(str))
        else:
            df['Stage_fear'] = stage_fear_encoder.transform(df['Stage_fear'].astype(str))

    if 'Drained_after_socializing' in df.columns:
        if fit:
            df['Drained_after_socializing'] = drained_encoder.fit_transform(df['Drained_after_socializing'].astype(str))
        else:
            df['Drained_after_socializing'] = drained_encoder.transform(df['Drained_after_socializing'].astype(str))

    # Drop non-feature columns if present
    for col in ['id', 'Personality']:
        if col in df.columns:
            df.drop(columns=col, inplace=True)

    # Impute missing values
    if fit:
        df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
    else:
        df_imputed = pd.DataFrame(imputer.transform(df), columns=df.columns)

    return df_imputed


In [97]:
# Train: fit=True
X_train = wrangle_data(df, fit=True)
y_train = LabelEncoder().fit_transform(df['Personality'])
# model = LogisticRegression(random_state=42)
model = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000))
])
model.fit(X_train, y_train)


# Test: fit=False (test_df has no Personality column)
test_df = pd.read_csv(r"C:\Users\T PLUG\Downloads\playground-series-s5e7\test.csv")
X_test = wrangle_data(test_df, fit=False)
predictions = model.predict(X_test)


In [98]:
result_df = pd.DataFrame({
    'id': test_df['id'],
    'Predicted_Personality': ['Extrovert' if p == 1 else 'Introvert' for p in predictions]
})

result_df

,id,Predicted_Personality
0,18524,Introvert
1,18525,Extrovert
2,18526,Introvert
3,18527,Introvert
4,18528,Extrovert
...,...,...
6170,24694,Introvert
6171,24695,Extrovert
6172,24696,Introvert
6173,24697,Introvert


In [99]:
# # Step 3: Model training
# X = wrangle_data(df, fit=True)
# y = LabelEncoder().fit_transform(df['Personality'])

# model = RandomForestClassifier(random_state=42)
# model.fit(X, y)

In [100]:
# # Step 4: Predict
# test_df = pd.read_csv(r"C:\Users\T PLUG\Downloads\playground-series-s5e7\test.csv")

# X_test = wrangle_data(test_df, fit=False)
# predictions = model.predict(X_test)

# # Optional: Convert 0/1 back to labels
# predicted_labels = ['Extrovert' if p == 1 else 'Introvert' for p in predictions]
# test_df['Predicted_Personality'] = predicted_labels


In [101]:
from sklearn.metrics import accuracy_score
y= pd.read_csv(r"C:\Users\T PLUG\Downloads\playground-series-s5e7\sample_submission.csv")
#accuracy = accuracy_score(y, result_df)
accuracy = accuracy_score(y['Personality'], result_df['Predicted_Personality'])

print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 25.28%


In [102]:
y

,id,Personality
0,18524,Extrovert
1,18525,Extrovert
2,18526,Extrovert
3,18527,Extrovert
4,18528,Extrovert
...,...,...
6170,24694,Extrovert
6171,24695,Extrovert
6172,24696,Extrovert
6173,24697,Extrovert


In [113]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

le = LabelEncoder()

# ---------- Wrangle Function ----------
def wrangle(df):
    df = df.copy()
    
    df['Stage_fear'] = le.fit_transform(df['Stage_fear'])
    df['Drained_after_socializing'] = le.fit_transform(df['Drained_after_socializing'].astype(str))
    #df['Personality'] = le.fit_transform(df['Personality'])  # Extrovert=1, Introvert=0

    # Drop columns that won't help
    if 'id' in df.columns:
        df.drop(columns=['id'], inplace=True)

    # Convert any object columns to category or encode
    for col in df.select_dtypes(include='object').columns:
        df[col] = pd.factorize(df[col])[0]
    
    # Fill missing values
    df.fillna(df.median(numeric_only=True), inplace=True)

    return df

# ---------- Load Data ----------
train_path = r"C:\Users\T PLUG\Downloads\playground-series-s5e7\train.csv"
test_path = r"C:\Users\T PLUG\Downloads\playground-series-s5e7\test.csv"
solution_path = r"C:\Users\T PLUG\Downloads\playground-series-s5e7\sample_submission.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
solution_df = pd.read_csv(solution_path)

# ---------- Preprocess ----------
X = wrangle(train_df.drop(columns='Personality'))
y = train_df['Personality']
X_test = wrangle(test_df)

# ---------- Feature Selection ----------
selector = SelectKBest(score_func=f_classif, k="all")
X_selected = selector.fit_transform(X, y)
X_test_selected = selector.transform(X_test)

# ---------- Train/Test Split for Validation ----------
X_train, X_val, y_train, y_val = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# ---------- Model ----------
model = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000, random_state=42, multi_class="auto"))
])

model.fit(X_train, y_train)
val_preds = model.predict(X_val)

# ---------- Evaluate on Validation ----------
val_acc = accuracy_score(y_val, val_preds)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

# ---------- Predict on Real Test Set ----------
test_preds = model.predict(X_test)
result_df = pd.DataFrame({'id': solution_df['id'], 'personality': test_preds})

# ---------- Compare with Sample Submission ----------
merged = pd.merge(result_df, solution_df, on='id', suffixes=('_pred', '_true'))
final_acc = accuracy_score(merged['Personality'], merged['personality'])
print(f"Test Set Accuracy: {final_acc * 100:.2f}%")

Validation Accuracy: 96.84%
Test Set Accuracy: 74.75%


c:\Users\T PLUG\anaconda3\envs\my_env\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [114]:
import joblib
joblib.dump(model, 'model.pkl')

['model.pkl']